In [1]:
from population_stats import *

nacc = load_dataset_as_df("NACC")
adni = load_dataset_as_df("ADNI")
ds = stack_datasets(nacc=nacc, adni=adni)

In [4]:
nacc

,age,mmse,sex,apoe,educ,progress,Dataset
